<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "C:\Users\shiwei\Documents\ImageAnalysis3\required_files\Startup_py3.py"
sys.path.append(r"C:\Users\shiwei\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

creating new log file
2022-03-07 15:37:14,036 [INFO] WRITING LOG OUTPUT TO C:\Users\shiwei\.cellpose\run.log
25160


In [2]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers'
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'mop_SE_1000')

In [3]:
# laod encoding
# summarize total readout usage
encoding_folder = os.path.join(pool_folder, f'Encoding_design')
region_2_bits = pickle.load(open(os.path.join(encoding_folder, 'region_2_bits.pkl'), 'rb'))

In [4]:
# load probes
probes = []
final_pb_filename = os.path.join(pool_folder, 'final_pool_probes_combinatory.fasta')

with open(final_pb_filename, 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        probes.append(record)


In [6]:
probes

[SeqRecord(seq=Seq('CCCGCAATGGCTGACAACCGTAGATAGAGACGTTCGTGAACTCCCTATTGACCT...TCC'), id='loc_4:105899094-105952080_gene_251_pb_60_pos_26508_strand_+_readouts_[NDB_487_c,NDB_628_c,NDB_629_c]_primers_[W1A03_primer_2,W1A10_primer_9]', name='loc_4:105899094-105952080_gene_251_pb_60_pos_26508_strand_+_readouts_[NDB_487_c,NDB_628_c,NDB_629_c]_primers_[W1A03_primer_2,W1A10_primer_9]', description='loc_4:105899094-105952080_gene_251_pb_60_pos_26508_strand_+_readouts_[NDB_487_c,NDB_628_c,NDB_629_c]_primers_[W1A03_primer_2,W1A10_primer_9]', dbxrefs=[]),
 SeqRecord(seq=Seq('CCCGCAATGGCTGACAACCGTAGATAGAGACGTTCGTGAAAAAGCTCTGGAATT...TCC'), id='loc_4:105899094-105952080_gene_251_pb_61_pos_26636_strand_+_readouts_[NDB_487_c,NDB_628_c,NDB_629_c]_primers_[W1A03_primer_2,W1A10_primer_9]', name='loc_4:105899094-105952080_gene_251_pb_61_pos_26636_strand_+_readouts_[NDB_487_c,NDB_628_c,NDB_629_c]_primers_[W1A03_primer_2,W1A10_primer_9]', description='loc_4:105899094-105952080_gene_251_pb_61_pos_26636_strand_

In [5]:
# load region names

region_2_names = {}

for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    #print(_reg,_gene)
    
    if _gene not in region_2_names:
        region_2_names[_gene] = _reg
print(region_2_names)

{'251': '4:105899094-105952080', '252': '4:114589240-114676242', '253': '4:115037700-115112192', '255': '4:115837369-115913327', '256': '4:116628906-116683403', '257': '4:117618081-117687368', '258': '4:123199825-123264995', '259': '4:123961238-124063077', '260': '4:124824301-124866678', '261': '4:126207621-126263017', '262': '4:126995025-127049608', '263': '4:127493306-127563000', '264': '4:127666510-127718932', '265': '4:127800701-127872488', '266': '4:129485757-129532804', '267': '4:130236140-130307752', '268': '4:130509810-130579025', '269': '4:131733908-131844723', '270': '4:132792079-132834284', '271': '4:135807186-135873467', '272': '4:137277244-137324456', '273': '4:148835951-148933937', '274': '4:155759707-155802098', '275': '5:3994952-4090244', '277': '5:4937946-5030393', '278': '5:5216972-5313712', '279': '5:15947115-16026399', '280': '5:19894500-19961425', '281': '5:21457453-21528145', '282': '5:24770084-24831106', '283': '5:25594458-25634378', '284': '5:27049822-27109916',

In [7]:
# laod encoding
# summarize total readout usage
region_2_readout_dict = pickle.load(open(os.path.join(library_folder, 'region_2_readout.pkl'), 'rb'))

In [8]:
# load used readouts
readout_usage_file = os.path.join(library_folder, 'readout_usage.pkl')
readout_dict = pickle.load(open(readout_usage_file, 'rb'))

In [9]:
readout_dict

{'c': [SeqRecord(seq=Seq('CCCTAATCCCGACTTTGTGGGTACGTTGAT'), id='NDB_376', name='NDB_376', description='NDB_376', dbxrefs=[]),
  SeqRecord(seq=Seq('TGCCACATCTAGGCGATACATGAGAGGCGT'), id='NDB_578', name='NDB_578', description='NDB_578', dbxrefs=[]),
  SeqRecord(seq=Seq('AGCCAGCTCGTAGCGCCTGAATTAGCTATA'), id='NDB_400', name='NDB_400', description='NDB_400', dbxrefs=[]),
  SeqRecord(seq=Seq('TCCGTTAAACGTCGGCTGCGTACTGGTACA'), id='NDB_533', name='NDB_533', description='NDB_533', dbxrefs=[]),
  SeqRecord(seq=Seq('TCTCAAGGTTACCGGTTGTCTATATCGTCT'), id='NDB_568', name='NDB_568', description='NDB_568', dbxrefs=[]),
  SeqRecord(seq=Seq('AGAGAGCGACTTGGGAACGCCTTCATATCC'), id='NDB_464', name='NDB_464', description='NDB_464', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCGTATCAGCGTGTACGGTCAAATACGAG'), id='NDB_466', name='NDB_466', description='NDB_466', dbxrefs=[]),
  SeqRecord(seq=Seq('GGACAGGACTGTTCACCGGAATACTTCTGA'), id='NDB_431', name='NDB_431', description='NDB_431', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCGAT

In [10]:
import pandas as pd
# generate the codebook
codebook = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
#codebook.add(['name', 'id'], axis=1)
codebook['id'] = list(region_2_readout_dict.keys())

In [11]:
codebook['name'] = [region_2_names[str(_r)] for _r in list(region_2_readout_dict.keys())]

In [12]:
codebook

,name,id,NDB_376,NDB_578,NDB_400,NDB_533,NDB_568,NDB_464,NDB_466,NDB_431,...,NDB_479,NDB_562,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604
0,1:9600471-9656458,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1:9749317-9845402,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1:20888605-20953251,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1:21475446-21564433,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1:21674557-21793163,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,X:101910335-102015609,987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
964,X:136772307-136851006,988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
965,X:140481839-140547985,989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
966,X:167122110-167191320,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in region_2_readout_dict.items()]) + 1

In [14]:
for _gname, _bits in region_2_readout_dict.items():
    binary_code = []
    for _i in range(max_bit):
        if f"c{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    print(_gname)
    codebook.loc[codebook['id']==_gname, codebook.columns[2:]] = binary_code
    #codebook.loc[codebook['name']==_gname,'id'] = int(_bits[0].split('u')[1])

1
2022-03-07 15:39:50,128 [INFO] Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-03-07 15:39:50,129 [INFO] NumExpr defaulting to 8 threads.
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
546
547
548
549
550
551
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
599
600
601
602
603
604
605
606
607
608
609
610
251
252
253
255
256
257
258
259
260
261
262
263
264
265
266
267


In [15]:
codebook

,name,id,NDB_376,NDB_578,NDB_400,NDB_533,NDB_568,NDB_464,NDB_466,NDB_431,...,NDB_479,NDB_562,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604
0,1:9600471-9656458,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1:9749317-9845402,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1:20888605-20953251,3,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1:21475446-21564433,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1:21674557-21793163,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,X:101910335-102015609,987,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
964,X:136772307-136851006,988,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
965,X:140481839-140547985,989,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
966,X:167122110-167191320,990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
chr_2_region_num = pickle.load(open(os.path.join(library_folder, 'chr_2_region_num.pkl'), 'rb'))

In [17]:
chr_2_region_num

{'4': 61,
 '5': 56,
 '6': 56,
 '7': 45,
 '8': 52,
 '9': 54,
 '10': 64,
 '11': 57,
 '12': 35,
 '13': 49,
 '1': 76,
 '2': 73,
 '3': 59,
 '14': 51,
 '15': 35,
 '16': 37,
 '17': 34,
 '18': 32,
 '19': 35,
 'X': 7}

In [18]:
chr_2_region_names = pickle.load(open(os.path.join(library_folder, 'chr_2_region_names.pkl'), 'rb'))

In [19]:
chr_2_region_names

{'4': [251,
  252,
  253,
  255,
  256,
  257,
  258,
  259,
  260,
  261,
  262,
  263,
  264,
  265,
  266,
  267,
  268,
  269,
  270,
  271,
  272,
  273,
  274,
  212,
  213,
  214,
  215,
  216,
  217,
  218,
  219,
  220,
  221,
  222,
  223,
  224,
  225,
  226,
  228,
  229,
  230,
  231,
  232,
  233,
  234,
  235,
  236,
  237,
  238,
  239,
  240,
  241,
  242,
  243,
  244,
  245,
  246,
  247,
  248,
  249,
  250],
 '5': [275,
  277,
  278,
  279,
  280,
  281,
  282,
  283,
  284,
  285,
  286,
  287,
  288,
  289,
  290,
  291,
  292,
  293,
  295,
  296,
  297,
  298,
  299,
  300,
  301,
  302,
  303,
  304,
  305,
  306,
  307,
  308,
  309,
  310,
  311,
  312,
  313,
  314,
  315,
  316,
  317,
  318,
  319,
  320,
  321,
  322,
  324,
  325,
  326,
  327,
  328,
  329,
  330,
  332,
  333,
  334],
 '6': [335,
  336,
  337,
  338,
  339,
  340,
  341,
  342,
  343,
  344,
  345,
  347,
  348,
  349,
  350,
  351,
  352,
  353,
  354,
  355,
  356,
  357,
  358,
  3

In [20]:
# generate region_to_chr_id
region_2_chr = {}
region_2_chr_order = {}
for _chr, _regions in chr_2_region_names.items():
    for _i, _region in enumerate(_regions):
        region_2_chr[_region] = _chr
        region_2_chr_order[_region] = _i

In [21]:
codebook['chr'] = [region_2_chr[_g] for _g in codebook['id']]
codebook['chr_order'] = [region_2_chr_order[_g] for _g in codebook['id']]

In [22]:
codebook.sort_values('id', inplace=True)

In [23]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP13-super-enhancers-1000_codebook.csv')

In [24]:
codebook.to_csv(save_filename,
                index=None)

# generate 500 gene codebook

In [79]:
codebook_500 = pd.DataFrame(columns=codebook.columns)

# extract 500 genes
sel_regions_500 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_500':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_500:
            sel_regions_500.append(_gene)
            
for _gene in sel_regions_500:
    codebook_500 = codebook_500.append(codebook.loc[codebook['id']==int(_gene)])
codebook_500.sort_values('id', inplace=True)
codebook_500

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0,1:3740000-3760000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
2,1:8740000-8760000,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
4,1:13740000-13760000,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,X:161240000-161260000,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52
437,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
437,X:166240000-166260000,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54
987,Y:1240000-1260000,1062,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,Y,0


In [80]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-500_codebook.csv')

codebook_500.to_csv(save_filename, index=None)

# 250-1

In [81]:
codebook_250_1 = pd.DataFrame(columns=codebook.columns)

# extract 250_1 genes
sel_regions_250_1 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_1':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_1:
            sel_regions_250_1.append(_gene)
            
for _gene in sel_regions_250_1:
    codebook_250_1 = codebook_250_1.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_1.sort_values('id', inplace=True)
codebook_250_1

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
1,1:6240000-6260000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
5,1:16240000-16260000,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5
9,1:26240000-26260000,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9
13,1:36240000-36260000,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13
17,1:46240000-46260000,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,X:116240000-116260000,1039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,37
424,X:131240000-131260000,1045,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,41
428,X:141240000-141260000,1049,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,45
432,X:153740000-153760000,1054,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,49


In [82]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_1_codebook.csv')

codebook_250_1.to_csv(save_filename, index=None)

# 250-2

In [83]:
codebook_250_2 = pd.DataFrame(columns=codebook.columns)

# extract 250_2 genes
sel_regions_250_2 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_2':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_2:
            sel_regions_250_2.append(_gene)
            
for _gene in sel_regions_250_2:
    codebook_250_2 = codebook_250_2.append(codebook.loc[codebook['id']==int(_gene)])
codebook_250_2.sort_values('id', inplace=True)
codebook_250_2

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
3,1:11240000-11260000,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3
7,1:21240000-21260000,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7
11,1:31240000-31260000,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11
15,1:41240000-41260000,16,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,15
19,1:51240000-51260000,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,X:121240000-121260000,1041,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,39
426,X:136240000-136260000,1047,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,43
430,X:146240000-146260000,1051,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,X,47
434,X:158740000-158760000,1056,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,51


In [84]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_2_codebook.csv')

codebook_250_2.to_csv(save_filename, index=None)